# QML: Quantum Machine Learning
#### _Ian Convy, Ruta Jawale, James Hulett, Alex Stennet, Adam Corbo_

This notebook documents the code (for Cirq 0.4.0) we used to construct and train our tree circuits.

In [13]:
# Requirements
import cirq
import tensorflow

## Motivation

The results of [1] demonstrate a method of constructing a quantum circuit that can be used to learn the handwritten digits of the MNIST dataset. One of the drawbacks of this model is that it learns the entries of arbitrary unitary matrices. This is impractical on actual quantum hardware, because the circuit would need to be recompiled after every update to the parameters. 

Our work explores an alternate implementation where we first decompose the unitaries as in [2] and then learn the parameters of this decomposition. Each gate in the decomposition has only one parameter, an angle of rotation, which is more feasible to update in practice.

## Cirq Circuit

Our quantum circuit construction is divided into 2 categories:
1. Image Encoding: takes an image and encodes it into a quantum circuit
2. Tree Construction: the full tree as is described in [1]

### Image Encoding

Creates a circuit which parameterizes an MNIST image using the particular pixel values in the interval $[0,1]$.

These values are encoded through the following mapping:
$$\left|0\right\rangle\mapsto\cos \left(\frac{\pi}{2}x_i\right)\left|0\right\rangle+\sin \left(\frac{\pi}{2}x_i\right)\left|1\right\rangle$$

Where $x_i$ is the $i$th pixel value

Of note, the gates are constructed for arbitrary image so that they can later be altered for new images without needing to recompile the entire cirtuit.

In [14]:
def get_image_gates(qubits, prefix = 'pixel'):
    """
    Create parameterized Ry gates for image encoding.

    Parameters:
        qubits (seq of qubits): Qubits to encode the images
        prefix (string): Prefix for Symbol names

    Returns:
        Sequence of gate operations on passed qubits
    """
    gates = []
    for (pos, qubit) in enumerate(qubits):
        name = '{}_{}'.format(prefix, pos)
        var = cirq.Symbol(name)
        gate = cirq.YPowGate(exponent = var)
        gates.append(gate(qubit))
    return gates

In [15]:
def get_param_resolver(flat_image, var_list = [], param_array = []):
    """
    Create ParamResolver from a flattened image and other optional parameters.

    This function creates a cirq.ParamResolver object from the pixel values of a flattened
    image, along with any other passed names/values. This allows for reuse of the circuit
    in different configurations.

    Parameters:
        flat_image (1-D numpy array): Array of pixel values
        var_list (sequence): Sequence of Symbol names
        param_array (sequence): Sequence of values to be assigned to the Symbols

    Returns:
        A ParamResolver object
    """
    pixel_dict = {'pixel_{}'.format(i) : flat_image[i] for i in range(flat_image.size)}
    param_dict = dict(zip(var_list, param_array))
    resolver = cirq.ParamResolver({**pixel_dict, **param_dict})
    return resolver

### Tree Construction

#### "Composition" Tree

Creates the tree structure specified in [1] in which every $2$ qubit gate is an arbitrary $4\times 4$ unitary.

The parameters of these gates are learned through the training process.

<img src="https://drive.google.com/uc?export=view&id=1OqhX52_VUAk4YDV9JYfbou040VnLodAB" alt="Drawing" style="width:30%"/>

We use specified parameters to first construct a hermitian matrix when is then exponentiated to turn it into a unitary matrix.

In [16]:
def build_unitary(gate_params):
    """
    Create 4x4 unitary matrix from passed parameters.

    Parameters:
        gate_params (sequence): Sequence of real numbers to parameterize the unitary

    Returns:
        A unitary matrix array 
    """
    diag_params = gate_params[:4]
    
    upper_triangle = np.zeros([4, 4], dtype = np.complex64)
    diag_matrix = np.diagflat(diag_params)
    
    off_diag_indices = [(i, j) for i in range(4) for j in range(i + 1, 4)]
    
    for (num, (i, j)) in enumerate(off_diag_indices):
        upper_triangle[i, j] = gate_params[4 + num] + gate_params[10 + num]*1j
    
    herm_matrix = upper_triangle + (upper_triangle.T).conjugate() + diag_matrix
    
    (eigvalues, eigvectors) = np.linalg.eigh(herm_matrix)
    
    eig_exp = np.exp(eigvalues * 1j)
    diag_exp = np.diagflat(eig_exp)
    
    unitary = np.einsum('bc,cd,de->be', eigvectors, diag_exp, eigvectors.T.conjugate())
    return unitary

This class contains all of the methods and attributes necessary to create and run the discriminative machine learning circuit specified in [1]. The tree must be rebuilt whenever the parameters are varied, but multiple images can be run on the same circuit.

In [17]:
class CompTree():
    """
    Circuit tree using two-qubit unitaries without decomposition.

    Attributes:
        num_pixels (integer): Number of pixels in the images
        qubits (list of qubits): Qubits for the tree
        image_gates (list of gate ops): Gates used to encode images
        circuit (cirq.Circuit): Circuit for the tree
        simulator (cirq.Simulator): Simulator for the tree
    """

    def __init__(self, num_pixels):
        """
        Initialize the tree with random-normal parameters.

        This method initializes the required number of qubits and the gates
        needed for the tree. The parameters are initialized on a normal distribution,
        and can be changed by calling 'build_tree'.

        Parameters:
            num_pixels (integer): Number of pixels in the images
        """
        self.num_pixels = num_pixels
        self.qubits = [cirq.LineQubit(pixel) for pixel in range(num_pixels)]
        self.image_gates = get_image_gates(self.qubits)
        num_params = (num_pixels - 1) * 16
        initial_params = np.random.normal(size = num_params)
        self.build_tree(initial_params)
        self.simulator = cirq.Simulator()

    def get_num_params(self):
        return (self.num_pixels - 1) * 16

    def build_tree(self, parameters):
        """
        Constructs the tree circuit using the specified parameters.

        This method constructs the 4x4 unitary gates needed for the tree along with
        a measurement gate for the prediction, and then compiles the image gates, 4x4
        gates, and measurment gate into one circuit. The two-qubits unitaries are
        optimized on directly, so the circuit must be recreated after each iteration.

        Parameters:
            parameters (1-D numpy array): Parameters used to build the 4x4 unitaries
        """
        op_size = 16
        tree_gates = []
        num_levels = int(np.log2(self.num_pixels))
        param_start = 0
        for level in range(num_levels):
            pair_gap = 2 ** level
            node_gap = pair_gap * 2
            for pos in range(0, self.num_pixels, node_gap):
                gate_params = parameters[param_start : param_start + op_size]
                unitary = mm.build_unitary(gate_params)
                gate = cirq.TwoQubitMatrixGate(unitary)
                op = gate(self.qubits[pos], self.qubits[pos + pair_gap])
                tree_gates.append(op)
                param_start += op_size
        self.circuit = cirq.Circuit()
        self.circuit.append(
            [self.image_gates, tree_gates],
            strategy = cirq.InsertStrategy.EARLIEST)

    def run(self, parameters, image_batch, reps):
        """
        Runs the circuit on the image batch.

        This method runs the tree circuit on each image of the batch, making the
        specified number of measurements. It returns an array with two columns giving
        the number of 'zeros' and 'ones' measured for each image.

        Parameters:
            image_batch (2-D array): Batch of images to be run
            reps (integer): Number of measurments to make on each image
            parameters: ignored input, just there to give a consistent API between tree types

        Returns:
            Array giving the results of the measurements for each image
        """
        resolvers = [get_param_resolver(image.flatten()) for image in image_batch]
        if reps:
            measure = cirq.measure(self.qubits[0], key = 'label')
            temp_circ = self.circuit.copy()
            temp_circ.append([measure], strategy = cirq.InsertStrategy.EARLIEST)
            results = self.simulator.run_sweep(
                temp_circ,
                params = resolvers,
                repetitions = reps)
            measure_counts = []
            for result in results:
                histo = result.histogram(key = 'label')
                count = [histo[0], histo[1]]
                measure_counts.append(count)
            probs = np.array(measure_counts) / reps
        else:
            results = self.simulator.simulate_sweep(
                self.circuit,
                params = resolvers,
                initial_state=0)
            probs = np.array([result.density_matrix([0]).diagonal().real for result in results])
        return probs

#### Decomposition Tree

Replaces the arbitrary $4\times 4$ unitary matrix from [1] as a decomposition of $Z$, $Y$, and $CNOT$ gates through the process specified in [2].

<img src="https://drive.google.com/uc?export=view&id=1ZzG1uve0m1UDXMu8aTd597GypTIdNG-l" alt="Drawing" style="width:100%;"/>

This function uses the ZY decomposition to represent an arbitrary single qubit unitary as a parametrized Z-Y-Z gate sequence. The names of the cirq.Symbol objects for the parameters are added to the passed variable list.

In [18]:
def zy_decomp(name, qubit, var_list):
    """
    Get parameterized Z-Y-Z gate operation sequence.

    Parameters:
        name (string): Label for the 'Symbol' objects
        qubit (cirq qubit): Qubit to be acted on
        var_list (list): List to add Symbol names

    Returns:
        List of gate operations on the passed qubit for the decomposition
    """
    z1_var = cirq.Symbol(name + '_z1')
    var_list.append(name + '_z1')
    z1 = cirq.ZPowGate(exponent = z1_var)
    y1_var = cirq.Symbol(name + '_y1')
    var_list.append(name + '_y1')
    y1 = cirq.YPowGate(exponent = y1_var)
    z2_var = cirq.Symbol(name + '_z2')
    var_list.append(name + '_z2')
    z2 = cirq.ZPowGate(exponent = z2_var)
    return [z1(qubit), y1(qubit), z2(qubit)]

This function uses the extended Cartan decomposition described in [arXiv:quant-ph/0307177](https://arxiv.org/abs/quant-ph/0307177) to represent an arbitrary two-qubit gate as a sequence of single-qubit unitaries and CNOT gates. These unitaries are then further decomposed using the ZY decomposition into gates that are implementable on actual quantum hardware.

In [19]:
def node_decomp(name, qubit_pair, var_list):
    """
    Get parameterized gate sequence for two-qubit node.

    Parameters:
        name (string): Label for the "Symbol" objects
        qubit_pair (sequence): Sequence of two qubits to act on
        var_list (list): List to add Symbol names

    Returns:
        List of gates that represent the two-qubit node
    """
    (q1, q2) = qubit_pair
    gates = []
    u1 = zy_decomp(name + "_u1", q1, var_list)
    v1 = zy_decomp(name + "_v1", q2, var_list)
    gates.append([u1, v1])
    cnot1 = cirq.CNOT(q1, q2)
    gates.append(cnot1)
    u2 = zy_decomp(name + "_u2", q1, var_list)
    v2 = zy_decomp(name + "_v2", q2, var_list)
    gates.append([u2, v2])
    cnot2 = cirq.CNOT(q1, q2)
    gates.append(cnot2)
    u3 = zy_decomp(name + "_u3", q1, var_list)
    v3 = zy_decomp(name + "_v3", q2, var_list)
    gates.append([u3, v3])
    cnot3 = cirq.CNOT(q1, q2)
    gates.append(cnot3)
    u4 = zy_decomp(name + "_u4", q1, var_list)
    v4 = zy_decomp(name + "_v4", q2, var_list)
    gates.append([u4, v4])
    return gates

This class contains all of the methods and attributes necessary to create and run the discriminative machine learning circuit using decompositions instead of full unitaries. The circuit only needs to be compiled once, with optimization done using Symbols.

In [20]:
class DecompTree():
    """
    Circuit tree made using universal gate set.

    Attributes:
        num_pixels (integer): Number of pixels in the images
        qubits (list of qubits): Qubits for the tree
        image_gates (list of gate ops): Gates used to encode images
        var_list (list): Symbol names used in the circuit
        circuit (cirq.Circuit): Circuit for the tree
        simulator (cirq.Simulator): Simulator for the tree
    """
    def __init__(self, num_pixels):
        """
        Initialized and compile the tree.

        This methon initializes the qubits and the simulator and then builds the tree
        by calling 'build_tree'.

        Parameters:
            num_pixels (integer): Number of pixels in the images
        """
        self.num_pixels = num_pixels
        self.qubits = [cirq.LineQubit(pixel) for pixel in range(num_pixels)]
        self.build_perm()
        self.simulator = cirq.Simulator()

    def get_num_params(self):
        return len(self.var_list)

    def build_perm(self):
        """
        Construct the parameterized gate operations for the tree.

        This method constructs the image gates and node decomposition gates needed for
        the tree, along with a measurement gate for the prediction, and then compiles the
        gates into one circuit. The gates are all parameterized using Symbols, so the
        circuit only needs to be compiled once.

        parameters is not used, and is only there to give a consistent API
        """
        self.image_gates = get_image_gates(self.qubits)
        tree_gates = []
        self.var_list = []
        num_levels = int(np.log2(self.num_pixels))
        for level in range(num_levels):
            pair_gap = 2 ** level
            node_gap = pair_gap * 2
            for pos in range(0, self.num_pixels, node_gap):
                name = '{}_{}'.format(level, pos)
                qubit_pair = [self.qubits[pos], self.qubits[pos + pair_gap]]
                node_gates = mm.node_decomp(name, qubit_pair, self.var_list)
                tree_gates.append(node_gates)
        self.circuit = cirq.Circuit()
        self.circuit.append(
            [self.image_gates, tree_gates],
            strategy = cirq.InsertStrategy.EARLIEST)

    def build_tree(self, parameters):
        pass

    def run(self, parameters, image_batch, reps):
        """
        Runs the circuit on the image batch using the specified parameters.

        This method runs the tree circuit on each image of the batch, making the
        specified number of measurements. It returns an array with two columns giving
        the number of 'zeros' and 'ones' measured for each image.

        Parameters:
            parameters (1-D array): Parameters used for the decomposition gates
            image_batch (2-D array): Batch of images to be run
            reps (integer): Number of measurments to make on each image

        Returns:
            Array giving the results of the measurements for each image
        """
        resolvers = [get_param_resolver(image.flatten(), self.var_list, parameters) for image in image_batch]
        if reps:
            measure = cirq.measure(self.qubits[0], key = 'label')
            temp_circ = self.circuit.copy()
            temp_circ.append([measure], strategy = cirq.InsertStrategy.EARLIEST)
            results = self.simulator.run_sweep(
                temp_circ,
                params = resolvers,
                repetitions = reps)
            measure_counts = []
            for result in results:
                histo = result.histogram(key = 'label')
                count = [histo[0], histo[1]]
                measure_counts.append(count)
            probs = np.array(measure_counts) / reps
        else:
            results = self.simulator.simulate_sweep(
                self.circuit,
                params = resolvers,
                initial_state=0)
            probs = np.array([result.density_matrix([0]).diagonal().real for result in results])
        return probs

## Data Preprocessing

We use Keras's built in MNIST dataset as our training and testing set.

Since we need a qubit for every pixel and have limited processing power, we scale each MNIST image to $4\times 4$. This is an acceptable downsampling size as it has been shown in standard learning practices that it is still possible to achieve a high accuracy on these images.

In [21]:
def load_data(keep_labels=[0,1]):
    mnist_data = tf.keras.datasets.mnist.load_data()
    
    train_images = mnist_data[0][0]
    train_labels = mnist_data[0][1]
    test_images  = mnist_data[1][0]
    test_labels  = mnist_data[1][1]

    subset_train = []
    for index in range(len(train_labels)):
        if train_labels[index] not in keep_labels:
            subset_train.append(index)
    subset_train_images = np.delete(train_images, subset_train, 0)
    subset_train_labels = np.delete(train_labels, subset_train, 0)

    subset_test = []
    for index in range(len(test_labels)):
        if test_labels[index] not in keep_labels:
            subset_test.append(index)
    subset_test_images = np.delete(test_images, subset_test, 0)
    subset_test_labels = np.delete(test_labels, subset_test, 0)

    return subset_train_images, subset_train_labels, subset_test_images, subset_test_labels

In [22]:
def resize_images(images, shape):
    num_images = images.shape[0]
    new_images_shape = (num_images, shape[0], shape[1])
    new_images = skimage.transform.resize(
        images,
        new_images_shape,
        anti_aliasing = True,
        mode = 'constant')
    return new_images

## Model Evaluation

Unlike ordinary machine learning algorithms, is isn't possible to exactly extract out the probabilities of a particular output using any physical means; instead, this would need to be done empirically via sampling many times. 

Because this is slow and can lead to inaccuracies in results, it also accomodates specifying no sampling (argument of 0 for `num_samples`) and instead taking advantage of the fact we are only simulating a quantum machine.

In [23]:
def find_empirical_probs(data, tree, tree_parameters, num_samples):
    """
    Empirically finds the probability of seeing 0/1 on each data point

    data: a numpy array of data points to be labeled
    tree: an instance of CompTree or DecompTree
    tree_parameters: a numpy array of the parameters needed to specify the tree to the classification function
    num_samples: the number of samples to take when estimating the label.  If 0, looks at the wave function.

    Returns a numpy array of dimension (len(data), 2) where ith row contains probabilities of classifying data point i as 0/1
    """
    tree.build_tree(tree_parameters)
    probs = tree.run(tree_parameters, data, num_samples)
    return probs

We use the max loss function specified in [1]:
$$Loss(\Lambda, x) = \max(p_\text{largest false}(\Lambda, x) - p_{l_x}(\Lambda, x) + \lambda, 0)^n$$

$$Loss(\Lambda) = \frac{1}{|\text{data}|}\sum_{x\in\text{data}}Loss(\Lambda, x)$$

In [24]:
def make_max_loss(data, labels, tree, lmda, eta, num_samples):
    """
    Create a loss function with the given hyperparameters and the data.

    data: a numpy array of data points to be labeled
    labels: numpy array of true classification labels
    lmda: hyperparameter
    eta: hyperparameter
    num_samples: number of samples to take when estimating labels.  If 0, looks at wave function instead.

    Returns a loss function that just takes in the parameters for the tree.
    """
    assert len(labels.shape) == 1

    def max_loss(tree_parameters):
        """
        Computes the max loss.

        tree_parameters: the current parameters to the tree

        Returns the loss of our model given the input parameters.
        """
        probs = find_empirical_probs(data, tree, tree_parameters, num_samples)
        assert len(labels) == probs.shape[0]

        correct_probs = probs[np.arange(len(labels)),labels]

        loss = np.sum(np.clip(1 - 2 * correct_probs + lmda, 0.0, None)**eta) / len(labels)
        
        return loss

    return max_loss

Simultaneous perturbation stochastic approximation (SPSA) is an optimization algorithm which, given a loss function and parameters to learn, attempts to find a global minimum by using a random perturbation vector to estimate the gradient.

We use this here to optimize the loss function specified in the above cell.

In [25]:
def update_params(curr_params, curr_velocity, loss_function, gamma, learning_rate, epsilon):
    """
    Do one round of SPSA. Runs loss_function twice.

    curr_params: the parameters to the tree before this round, as a numpy array
    curr_velocity: the velocity from the last round, as a numpy array
    loss_function: the loss function being used
    gamma: hyperparameter controlling how quickly velocity decays
    learning_rate: hyperparameter controlling how big our updates are
    epsilon: hyperparameter controlling the size of the perterbation

    Returns the parameters and velocity after this step, as a numpy arrays.
    """
    perturbation = (2*np.round(np.random.random(len(curr_params))) - 1) * epsilon

    loss_plus = loss_function(curr_params + perturbation)
    loss_minus = loss_function(curr_params - perturbation)
    approx_gradient = (loss_plus - loss_minus) / (2 * perturbation)
    next_velocity = gamma * curr_velocity - learning_rate * approx_gradient
    next_params = curr_params + next_velocity
    return next_params, next_velocity

This function pieces together all of the components by iterating for as many epochs as are specified and uses the loss function to update the circuit parameters on each iteration. 

In [26]:
def train_model(train_data, train_labels, val_data, val_labels, hyper, tree_class=CompTree):
    """
    Trains the model.

    train_data: a numpy array (batch_size, image dimension) of training data to be classified
    train_labels: a numpy array (batch_size) of corresponding labels
    val_data: a numpy array (batch_size, image dimension) of validation data to be evaluated
    val_labels: a numpy array (batch_size) of corresponding labels
    hyper: a dictionary containing hyperparameter values.
    tree_class: which type of tree to use.  Either CompTree or DecompTree

    Returns tree parameters.
    """
    # Unpack training hyperparameters
    batch_size, num_epoch = hyper['batch_size'], hyper['num_epoch']
    # Unpack update hyperparameters
    a, s, A, b, t, gamma = hyper['a'], hyper['s'], hyper['A'], hyper['b'], hyper['t'], hyper['gamma']
    # Unpack loss hyperparameters
    lmda, eta = hyper['lmda'], hyper['eta']
    # Unpack sampling hyperparameter
    num_samples = hyper['num_samples']

    # Initialize the params and velocity
    num_iter = len(train_data) // batch_size
    tree = tree_class(len(train_data[0]))
    mu, sigma = 0, 1
    params = np.random.normal(mu, sigma, tree.get_num_params())
    velocity = 0

    for epoch in range(num_epoch):

        for i in tqdm(range(num_iter)):
            loss_function = make_max_loss(train_data[i*batch_size:(i+1)*batch_size],
                                          train_labels[i*batch_size:(i+1)*batch_size],
                                          tree, lmda, eta, num_samples)

            epsilon = a/(epoch+1+A)**s
            learning_rate = b/(epoch+1)**t
            params, velocity = update_params(params, velocity, loss_function,
                                                    gamma, learning_rate, epsilon)

        print("-------------", flush=True)
        print("Epoch: {}".format(epoch), flush=True)
        # Evaluate model
        accuracy = evaluate_model(val_data, val_labels,
                                  params, num_samples, tree_class)
        print("Validation Accuracy: {}".format(accuracy), flush=True)

    return params

Used to calculate the accuracy of a particular model

In [27]:
def evaluate_model(data, labels, tree_parameters, num_samples, tree_class=CompTree):
    """
    Evaluates the model on a set of data.

    data: a numpy array (num images, image dimension) of data points to be labeled
    labels: a numpy array (num images) of corresponding labels
    tree_parameters: the parameters to build the tree with
    num_samples: how many times we sample when estimating labels.  If 0, uses wave function instead of sampling
    tree_class: what type of tree to use.  Either CompTree or DecompTree

    Returns the percentage of data points correctly labeled.
    """
    tree = tree_class(len(data[0]))
    prediction_probs = find_empirical_probs(data, tree, tree_parameters, num_samples)
    prediction = np.argmax(prediction_probs, axis=1)
    return (100.0 * np.count_nonzero(prediction==labels)) / len(labels)

In [ ]:
# Load images with labels 0 and 1
train_images, train_labels, test_images, test_labels = load_data(keep_labels=[0,1])

# Split training and validation set 80/20
len_train, len_test = len(train_labels), len(test_labels)
len_train, len_val = len_train - int(0.2*len_train), int(0.2*len_train)
train_images, val_images = train_images[:len_train], train_images[len_train:]
train_labels, val_labels = train_labels[:len_train], train_labels[len_train:]

# Preprocess images
new_shape = (4,4)
train_data = resize_images(train_images, new_shape).reshape((len_train, new_shape[0]*new_shape[1]))
val_data = resize_images(val_images, new_shape).reshape((len_val, new_shape[0]*new_shape[1]))
test_data = resize_images(test_images, new_shape).reshape((len_test, new_shape[0]*new_shape[1]))

There are 11 different hyper parameters that contribute to the models learning in a handful of way:
- batch size, how many images are put into a mini-batch, is controlled by `batch_size`
- how many epochs are run is controlled by `num_epoch`
- perterbation annealing is updated through 3 hyper parameters `a`, `A`, `s` and, given an epoch iteration `k`, is updated via the following multiplier: 
$$\frac{a}{(k + 1 + A)^s}$$
- learning rate annealing is updated through 2 hyper parameters `b`, `t` and, given an epoch iteration `k`, is updated via the following multiplier: 
$$\frac{b}{(k + 1)^t}$$
- how fact the velocity decays is controlled by `gamma`
- the threshold for how far apart the correct / incorrect label probabilities should be is controlled by `lmda`
- the loss is raised to an exponent which is controlable via `eta`, this allows for further specificity of how the loss function contributes (a larger exponent will make the loss smaller)
- how many samples are used to estimate a label probability is controlled by `num_samples` (note: setting this to 0 will cause the model to use the exact wave function instead which isn't feasible on actual quantum hardware)

In [ ]:
# Select hyperparams
hyper = {"batch_size": 200, "num_epoch": 10, "num_samples": 0, "lmda": 0.234, "eta": 4.59,
         "a": 10.0, "s": 4.13, "A": 10, "b": 10.0, "t": 0.658, "gamma": 0}

# Select training, validation, and test subset
num_data = 2000
train_data, train_labels = train_data[:num_data], train_labels[:num_data]
val_data, val_labels = val_data[:num_data], val_labels[:num_data]

# Train tree model on training dataset
tree_parameters = train_model(train_data, train_labels, val_data, val_labels,
                              hyper=hyper, tree_class=CompTree)

# Validate tree model on training, validation, and test datasets
train_acc = evaluate_model(train_data, train_labels, tree_parameters, hyper["num_samples"])
val_acc = evaluate_model(val_data, val_labels, tree_parameters, hyper["num_samples"])

print("Training: {}% Validation: {}%".format(train_acc, val_acc), flush=True)

## Citations

[1] W. Huggins, P. Patel, K. B. Whaley, and E. M. Stoudenmire, Towards Quantum Machine Learning with Tensor Networks

[2] Vidal, Guifre, and Christopher M. Dawson. Universal quantum circuit for two-qubit transformations with three controlled-NOT gates. Physical Review A 69.1 (2004): 010301.